# Optimized Tumor Segmentation with EfficientNetB7

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

model = smp.Unet(
    encoder_name="efficientnet-b7",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
)

dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device, dtype=dtype)

criterion = smp.losses.DiceLoss(mode='binary') + nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

def predict(model, image):
    model.eval()
    with torch.no_grad():
        image = transform(image=image)['image'].unsqueeze(0).to(device, dtype=dtype)
        output = model(image)
        return torch.sigmoid(output).cpu().numpy()

print("Model optimized with EfficientNetB7 and mixed precision!")
